In [ ]:
# Import the X-HEEP Python class
from pynq import x_heep

In [ ]:
# Load the X-HEEP bitstream
x_heep = x_heep()

In [ ]:
# Compile the application
x_heep.compile_app("virtual_adc_read")

In [ ]:
# Init the ADC memory
adc = x_heep.init_adc_mem()

In [ ]:
# Reset the ADC memory
x_heep.reset_adc_mem(adc)

In [ ]:
# Write the ADC memory
x_heep.write_adc_mem(adc)

In [ ]:
# Run the application
x_heep.run_app()

In [ ]:
# Delete adc
del adc